# Thursday, May 23, 2024

In [ ]:
# I think the thing to do, if I can, is to model it on the imputed data
# then model it again on the subsample where I've dropped all these NAs
# at least the ACE NAs, where there's really a lot.
# could also try modeling it just without those columns
# a random forest would probably be good, 
# make sure the whole thing isn't hinging on imputed data
# Meyer also probably ran some tests in his paper to make sure this is ok

I also found a TON of really interesting features that I had to drop just to make sure I have a straightforward enough project to pull off in 2 weeks (left).  I'd love to go back and tinker with them for my portfolio though!

\# script 3 in full: 2024-05-29_notes_about_non-scale_columns.txt

notes about non-scale columns

- for right now I've kept the current well being and the future predicted well being columns, because I think it could be interesting to see if predicting higher or lower than current correlates with anything.  I've also made instructions to combine them (higher is better, regardless fo hether it's current WB or optimism for future WB), which may end up being more parsimonious.
- I've also kept Q3, which is about happiness, but I suspect it's very corr'd with WB.  check that, and then either combine or drop it.


# Monday, June 3, 2024

In [ ]:
# Friends of gmilesaway2
# It's actually not a very good assumption that they live
# more than 60 miles away. 73.5% of people live w/i 60m!
# And yet, most people have never gone.  
# I think the thing to impute here is a 0 under the original 
# meaning, that is, within 60 miles.

# Wednesday, June 5, 2024

I think when this is all over, I should reload a clean, fresh copy of the dataframe and see
- how many NAs there are per ROW -- I wonder if there are a handful of people who skipped a lot of questions and I've ended up imputing basically their whole row
- how many cells I've altered.  it all seems so reasonable when I'm going one column at a time, but when I step back and watch every single column turn into an _ei column, it makes me wonder if I'm destroying my dataset.  If the final tally is staggering, then I should probably go with Hank's suggestion of only picking an handful of columns and ignoring the rest.

In any case, today I finished doing the imputation as laid out in imputation_plan.xlsx.  Or, mostly.  I changed my mind on a few things as I went, and discovered things along the way.  Most memorably, some column that I meant to impute with the median populated value ended up just being 0s, because once I converted the 97s to 0s, that was the median.  I'm sure there's a way to tell SI to impute only from values in a range, but 0 was faster.  I also cleaned up a bunch more of those 97s et al., and did some subset investigations.  On some poverty and employment measures there were probably more columns that I could have used to predict the missing values, but using measures of central tendency or just the columns that came most readily to mind was fastest, and will have to do for this iteration.

In [ ]:
# BUT because I imputed the 97s to be 0s, and there were so many 97s, that made the 
# median and mode 0.  So most of these guys just ended up imputed with 0s.
# It's not exactly what I wanted (the true missings are people who DID have the 
# relevant thoughts, but just wouldn't say when), but given that there are so few 
# true missings, and that this has already eaten up a ton of time, I think it'll
# be fine to leave for this iteration.  Put it in the next steps.

# Thursday, June 6, 2024

**I discovered that the kessler values *are not normally distributed*.**  I tried a few different things, and taking the square root of them showed the most improvement, although the qqplot still looks not-great (not as bad as project 4 though!).  **I created a column for the square root of these values, but *did not delete the original values*.  It will be important to not include them, or the `studyid` in my X matrix!**

All of the non-NA NAs were handled a little differently, but (I think) I sufficiently commented the code to explain what I did and why.  Where possible/necessary/I remembered, I tried to mirror what I did with the actual NAs when dealing with these values.

I then went through all the columns indicated in my `feat_eng_dict`, and recoded or transformed them as necessary.  I also updated the names of the columns in the dictionary, because all but literally 1 of them have had `'_ei'` and sometimes `'_r'` tacked onto them by now.  What I did with each batch of columns is marked in comments in the code.  This is also the script where I defined and used a function.  Also in this script, I OHE'd the columns about what religion someone was raised in or practices now.  I first collapsed these categories into fewer bins, then OHE'd them.  I chose to drop the resulting column that corresponded to, more or less, "not religious," because it seemed most intuitive.  The other 2 categories are for Christian-ish religions, and other.  Despite it being somewhat disrespectful, I chose to collapse all other religious practices into "other" because these categories were all so small on their own.  Even combined, they are dwarfed by the other two options.

### script 3 in full:
2024-06-06_y-variable.py

In [ ]:
# Thursday, June 6, 2024

# I should probably also choose a y variable.  [several hours latere]  Let's go with
# Kessler6!  It's a general mental health scale.  

# Ok, I'll square root it.

### script 4 in full:
2024-06-06_column_recoding_for_feat_eng_dict_combinations.py

In [ ]:
# Composite Scale Calculations

# In this script, I need to go back to `feat_eng_dict`, that list I made in 
# `2024-05-23_thru_2024-05-30_exploring_Meyer_2023_dataset.py`, and use it to 
# combine some of these columns into composite columns and drop the rest.



# religiosity 

# here's what 179 is now; 1 and 2 are massive in 180
# 1 Protestant (for example, Baptist, Methodist) 295 19.4 %
# 2 Roman Catholic 133 8.8 %
# 3 Mormon (Church of Jesus Christ of Latter-day Saints or LDS) 10 0.7 %
# 4 Orthodox (Greek, Russian, or another Orthodox church) 6 0.4 %

# 5 Jewish 38 2.5 %
# 6 Muslim 3 0.2 %
# 7 Buddhist 30 2.0 %
# 8 Hindu 1 0.1 %
# 11 Spiritual 262 17.3 %
# 12 Something else 95 6.3 %

# 9 Atheist (do not believe in God) 192 12.6 %
# 10 Agnostic (not sure if there is a God) 156 10.3 %
# 13 Nothing in particular 273 18.0 %

# collapse 3-8 into "other organized"
# .... actually collapse more, bcz this needs to be OHE'd

# 1-4 christian-influenced religious     1
# 5-8, 11-12  non-christian religious    5
# 9-10, 13 not religious                 make this 0, because it's literally none



# I actually cannot think of a meaningful way to combine these.I think the 
# thing to do is just leave them all in alone, then see what pops as meaningful.

# Friday, June 7, 2024

Today I finally got to run my `feat_eng_dict`!  I made some changes to it, and just summed up a lot of the things I was originally going to binarize.  I wanted to get a good look at them to see if there were differences based on the extent to which whatever the thing is has happened to someone, and in a handful of cases there were, but for the most part I think I would have been better off binarizing them.  I will try to do that in the future.  **Throughout the course of this evening, I have been thinking that I have to treat all of these columns the same way.  At this moment, I no longer think that's true.**  I think it's probably fine to say, "I looked at these guys, and some of them had magnitude effects and some of them didn't.  I binarized the ones that didn't, and kept the others."

I then conducted some exploratory data analysis, mostly through the production of graphs.  These graphs can be found in the images folder.  In addition to the frequency distributions of each variable and their relationship (shown through scatterplots) with the target variable, I also created qqplots for each variable, and for 3 transformed versions of each variable.  The purpose of this was to assess each variable's normality, and determine whether any of them would benefit from a linear transformation.  I ultimately decided that this was not likely to useful for any variable other than the target, which is the square root of the Kessler scores.

Look at these plots also revealed a number of variables that still need to be one-hot encoded, 0-based, or otherwise modified.  I would also like to rename the columns to be more intuitive, so that I don't have to contiually refer to the documentation.  I opted not to do all that before running my first model, but it would be good to do it before including any of those variables in the model.  I started several lists to do these cleaning tasks, but did not finish any of them.  These lists can be found in `2024-06-07-eda.py`, `2024-06-07_NEW_feat_eng_dict.py`, and `2024-06-07_columns-i-want-to-screw-with-some-more-and-how.py`.  When I have time to address these cleaning issues, I intend to rename and modify these scripts.

Finally, I created my first model!  I flipped through the scatterplots and just eyeballed some predictors that seemed reasonable, and put them all into a linear regression.  I conducted a train-test split for a little bit of a check on the model, to get a sense of how overfit it might be, but I kept the test size small because this model is mostly for inference.  I am not concerned about its applicability to new data in the future, because newer data is likely to be legitimately different than this data from 2016.  I do, however, want to keep an eye on overfitting, especially as I add more predictors to my model.  Dimensionality has been a concern throughout this entire process, and I am wary of allowing my model to coil itself too tightly around the data.  Although I am not concerned about this model generalizing perfectly to future data, I also do not want it to be so bespoke to the current data that it becomes unsuitable for interpretation.  In other words, if it becomes so overfit that it cannot even generalize to testing data from the same dataset, then its coefficients are unlikely to be meaningful for any level of analysis.

I did some cross validation because it seemed like a good idea to demonstrate that I know how to do that, although without gridsearching over anything, I'm not sure it's really necessary.

I used several metrics to evaluate my model, and I think it's a decent model for the area.  $R^2$ was around 0.6 for both training and testing, and all of the loss functions were <1.  (This makes sense, because the y-variable itself is a square root.  Squaring a float <1 will make it smaller, not larger.)  This implies that my model is usually within a range of +/-1 with its predictions of the square root of Kessler scores (range: 0-4.9), and within +/-0.5 for the re-squared values (range: 0-24).

I honestly can't believe how good this thing is for being so quickly slapped together.

### script 4 in full:
4. 2024-06-07-eda.py


In [ ]:
# Friday, June 7, 2024
# EDA

# As usual, we're starting with my beloved autoplots - now with new arguments!

autoplots(meyer, 'w1kessler6_i', qqplots=True, transform=True, line=False, verbose=True)
autoplots(meyer, 'kessler6_sqrt', qqplots=True, transform=True, line=False, verbose=True)

# There are a few more that I need to OHE, 0-base, or otherwise tinker with, but I'm going
# to try to resist the urge to do that until I have SOME kind of model fit.

### script 5 in full:
5. 2024-06-07_NEW_feat_eng_dict.py


In [ ]:
# Thursday, June 7, 2024
# NEW feat_eng_dict

# I thought I was so smart before, just getting a sum from these guys.
# Fool.  It dilutes the power of the "yes"s! There aren't enough "yes"s 
# in any one level of frequency to compete with the "no"s.

# these are some visually very promising ones
good_ones = []

### script 7 in full:
7. 2024-06-07_modeling-part-1.py


In [ ]:
# Friday, June 7, 2024
# Modeling!


# I'm just going to eyeball some features from the scatterplots
good_ones = ['chronic_strain', 'suicidality', 'w1age', 'w1auditc_i', 
    'w1connectedness_i', 'w1conversion', 'w1dudit_i', 'w1everyday_i', 
  'w1feltstigma_i', 'w1idcentral_i', 'w1internalized_i', 'w1lifesat_i', 
  'w1meim_i', 'w1pinc_i', 'w1poverty_i_ei', 'w1q03_ei', 'w1q33_ei', 
  'w1q52_ei', 'w1q72_ei', 'w1q74_22_ei', 'w1q74_21_ei', 'w1q140_ei', 
  'w1q166_ei', 'w1q167_ei', 'w1q171_8_ei', 'w1q181_ei_r', 'w1socialwb_i', 
  'w1socsupport_i']

# Very small testing set because this is an inferential model